# CNN mit Keras für CIFAR-10  

## Problemstellung - 10 Objektklassen

Die Aufgabe, Objektklassen anhand eines kleinen Farbbildes zu erkennen, ist ein sehr komplexes Problem für einen Computer. Für ein solches Problem sind CNN gut geeignet.

## Notwendige Libraries + Setup GPU

Die Berechnungen dieses Dokuments sind rechenaufwändiger. Daher bietet es sich an eine Funktion von Google Colab zu nutzen. Eine GPU kann über

*   Edit -> Notebook Settings -> Hardware acceleration -> GPU

ausgewählt werden (ist aber auch der Default).

In [ ]:
import numpy as np
import keras as keras
from keras.datasets import cifar10
#%tensorflow_version 1.x

## CIFAR Data Set & Preprocessing

Wir laden die Daten und plotten von jeder Klasse das erste Bild als Beispiel 


In [ ]:
(xTrain, yTrain), (xTest, yTest) = cifar10.load_data() 
noOfClasses = 10
im = []
import matplotlib.pyplot as plt
fig = plt.figure()
for i in range(noOfClasses):
    ax = fig.add_subplot(2, 5, i+1, xticks=[], yticks=[])
    first = np.flatnonzero(yTrain == i)[0] 
    im.append(xTrain[first,:,:,:])
    ax.set_title(i)
    ax.imshow(im[i])
plt.show()

## Normalisieren und Formatieren der Daten

Nun normieren wir jedes Input-Bild auf [0,1].

Schließlich transformieren wir den Output (jeweils eine Ziffer 0-9) in ein **"one-hot-encoding"**, also einen binären Vektor der Dimension 10 mit einer 1 bei der richtigen Ziffer.

In [ ]:
from keras.utils import np_utils
YTrain = np_utils.to_categorical(yTrain, noOfClasses)
YTest  = np_utils.to_categorical(yTest, noOfClasses)
XTrain = xTrain/255.0
XTest  = xTest/255.0

XTrain.shape
YTrain[0,:]

## Trainieren des CNN mit Keras

In [ ]:
# Die Lernrate beeinflusst wie stark die Werte bei einem Parameterupdate
# verändert werden. Zu große Lernraten führen zu Oszillationen und
# zu kleine Lernraten sorgen für ein zu langsames Lernen.
learning_rate = 0.001

# Die batch_size ist die Anzahl der Trainingsbeispiele, für die die Deltas akkumuliert
# werden, bevor ein Parameter-Uupdate durchgeführt wird.
batch_size = 64 # 32 #128

# Die Anzahl der Epochen bestimmt wie oft die gesamten Daten gelernt werden
# sollen. Bei zu wenig Epochen hat das Netz noch nicht konvergiert. Bei
# zu vielen Epochen wird das Netzwerk stagnieren und im schlimmsten Fall
# die Trainingsdaten auswendig lernen.
epochs = 20

Aufbau des CNN

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

l2Reg = 0.001 #0.001

CNN = Sequential()
CNN.add(layers.Conv2D(32,(3,3),padding='same',activation='relu',kernel_regularizer=l2(l2Reg),
                      input_shape=(32,32,3)))
CNN.add(layers.MaxPool2D(pool_size=(2, 2),padding='same'))
CNN.add(layers.Conv2D(32,(3,3),padding='same',activation='relu',kernel_regularizer=l2(l2Reg)))
CNN.add(layers.MaxPool2D(pool_size=(2, 2),padding='same'))
CNN.add(layers.Conv2D(64,(3,3),padding='same',activation='relu',kernel_regularizer=l2(l2Reg)))
CNN.add(layers.MaxPool2D(pool_size=(2, 2),padding='same'))
CNN.add(layers.Conv2D(64,(3,3),padding='same',activation='relu',kernel_regularizer=l2(l2Reg)))
CNN.add(layers.MaxPool2D(pool_size=(2, 2),padding='same'))
CNN.add(layers.Flatten())
### Aufgabe
### ??? CNN.add(????)
### ??? CNN.add(????)
CNN.summary()
CNN.compile(optimizer=Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])


$\color{red}{\mbox{Programmieraufgabe}}$: Vervollständigen Sie das Netz hinter `layers.Flatten()`: Ergänzen Sie einen Hidden Layer mit 512 Neuronen und ReLU-Aktivierung, gefolgt von einem Hidden Layer mit 256 Neuronen mit ReLU-Aktivierung und schließlich einen Output-Layer (welche Größe?) mit einer zum Output passenden Aktivierung (welcher?)



Trainieren des CNN:

In [ ]:
CNN.fit(XTrain,YTrain,epochs=epochs,batch_size=batch_size)

Evaluieren des CNN auf dem Test-Set:

In [ ]:
yPred = CNN.predict(XTest)
choice = np.argmax(yPred, axis=1)

confusionMatrix = np.zeros((noOfClasses,noOfClasses))
for i in range(noOfClasses):
    index = np.flatnonzero(yTest == i)
    for j in range(noOfClasses):
        index2 = np.flatnonzero(choice[index] == j)
        confusionMatrix[i,j] = len(index2)
print(confusionMatrix)

print("\n")
scores = CNN.evaluate(XTest,YTest,batch_size=64)
print("Accuracy Test Set: %.2f%%" % (scores[1]*100))


$\color{red}{\mbox{Aufgabe}}$: 

1.   Welche Test-Set Accuracy erreichen Sie?
2.   Wie viele Pferde werden irrtümlich als Flugzeuge erkannt?
3.   Visualisieren Sie (mithilfe der nachfolgenden Notebook-Zelle) das Filter-Mosaik für ein Auto-Bild und den 1. und 2. Faltungslayer. Interpretation?



## Output-Mosaic einiger Filter

In [ ]:
from keras import backend as K

# Parameter:
#    imSingle: ein einzelnes Input-Bild, für das die Filter-Antworten gezeigt werden
#    outLayer: von welchem CNN-Layer 1,2,3, ... die Filter-Antworten gezeigt werden 
def outputMoasic(imSingle, outLayer):
    pic = imSingle[np.newaxis,...]
    
    outputSingleLayer = K.function([CNN.layers[0].input],[CNN.layers[outLayer].output])
    picFilter = outputSingleLayer([pic])[0]
    
    gridy = 8
    gridx = 4 if outLayer < 4 else 8
    size = picFilter[0,:,:,0].shape[0] 
    mosaic = np.zeros( (gridx*size,gridy*size))
    
    for l in range(0,picFilter.shape[3]):
        x = int(np.floor(l / gridy))
        y = l%gridy
        mosaic[x*size:(x+1)*size,y*size:(y+1)*size] = picFilter[0,:,:,l]
    plt.figure()
    plt.imshow(mosaic,cmap='binary')
    plt.show()

#### Aufgabe 
#??? outputMosaic(???,???)
#??? outputMosaic(???,???)



## Die falsche Accuracy
$\color{red}{\mbox{Programmieraufgabe}}$: Wenn man sich den Start der Arrays yTest und choice anschaut

*   yTest =  [3 8 8 0 6 6 1 6 3 ...]
*   choice= [3 8 8 0 6 6 1 6 3 ...]

so würde man meinen, dass `np.mean(yTest==choice)` auch die Testset-Accuracy berechnen würde. Finden Sie heraus, warum das falsch ist! Finden Sie eine Code Zeile, die  unter Verwendung von `yTest` und `choice` die richtige Accuracy berechnet:

`??? accuracy = ... yTest ... choice ...`







In [ ]:
# ???

$\color{red}{\mbox{Programmieraufgabe}}$: Bauen Sie den Parameter Lernrate in das CNN ein und testen Sie verschiedene Werte.

Stellen Sie fest, wie viele Parameter das Netz in jedem Layer bzw. insgesamt hat.



$\color{red}{\mbox{Programmieraufgabe}}$: Ändern Sie auf `loss='mse'` und dokumentieren Sie, was passiert.



## Vergleich mit MNIST-DNN
$\color{red}{\mbox{Programmieraufgabe}}$: Kopieren Sie die DNN-Architektur aus `mnist_sequential.ipynb` ein (vielleicht mit Aktivierung `softmax` im Output-Layer) und wenden Sie sie auf CIFAR-10 an. (ACHTUNG: Sie müssen die Input-Daten passend reshapen!)

Welche Accuracy erreichen Sie mit DNN im Vergleich zum CNN?

## Zum weiteren Experimentieren
$\color{red}{\mbox{Optionale Aufgabe}}$: Variieren Sie weitere Parameter (Batch Size, Epochen, Regularisierungskoeffizient) oder Elemente der Architektur (Aktivierungsfunktionen einzelner Layer, welche Layer Regularisierung haben, ..., andere CNN-Layer, ..., evtl. Einfügen eines Dropout-Layers, ...) und dokumentieren Sie, was passiert. 